# Report for Banana Collector solved by a DQN algorithm
## Code implementation description
Files and how it works
## Learning algorithm
Description, hyperparameters and neural networks architectures
## Plots of rewards
Reports the number of episodes needed to solve the environment
## Ideas for future work
The submission has concrete future ideas for improving performance